In [ ]:
from SystemConfig import Path
from asyncio import Queue
import Workspace.Utils.BaseUtils as base_utils
from Workspace.Services.PrivateAPI.Trading.FuturesTradingClient import FuturesTradingClient as futures_tr_client
from SystemConfig import Path


class Wallet:
    __slots__ = (
        "init_balance",   # 최초 입금한 자금
        "total_balance",  # 현재 총 자산 (예수금 + 평가금액)
        "free_balance",   # 즉시 거래 가능한 금액
        "lock_balance",   # 미체결 주문으로 잠긴 금액
        "margin_balance", # 포지션 유지에 필요한 증거금
        "pnl_balance",    # 손익 금액 (total_balance - init_balance)
        "unrealized_pnl_balance",  # 미실현 손익
        "pnl_ratio",      # 손익률
        "queue",          # 작업 큐
        "futures_tr_client"  # API 클라이언트
    )

    def __init__(self, init_balance: float, queue: Queue, futures_trading_client: futures_tr_client):
        self.init_balance: float = init_balance
        self.total_balance: float = self.init_balance
        self.free_balance: float = self.init_balance
        self.lock_balance: float = 0
        self.margin_balance: float = 0
        self.pnl_balance: float = 0
        self.unrealized_pnl_balance: float = 0
        self.pnl_ratio: float = 0
        self.queue = queue
        self.futures_tr_client = futures_trading_client

    async def run_update(self):
        while True:
            # 올바른 변수명 사용
            n = await self.queue.get()
            print(f" {n} 회차 \n")
            print(self)
            await self.update_balance()
            self.queue.task_done()

    async def update_balance(self):
        """
        🐣 API에서 계좌 잔고를 가져와 Wallet 정보를 업데이트하는 함수
        """
        data = await asyncio.to_thread(self.futures_tr_client.fetch_account_balance)

        self.total_balance = float(data["totalMarginBalance"])
        self.margin_balance = float(data["totalMarginBalance"])
        self.free_balance = float(data["availableBalance"])
        self.lock_balance = float(data["totalOpenOrderInitialMargin"])
        self.unrealized_pnl_balance = float(data["totalUnrealizedProfit"])
        self.pnl_balance = float(data["totalCrossUnPnl"])

        # ZeroDivisionError 방지
        self.pnl_ratio = (self.pnl_balance / self.init_balance) if self.init_balance != 0 else 0
    
    async def send_queue(self):
        n = 0
        await self.update_balance()
        while True:
            await self.queue.put(n)
            n +=1
            await asyncio.sleep(5)


    def __repr__(self):
        return (
            f"1. seed money: {self.init_balance}\n"
            f"2. total balance: {self.total_balance}\n"
            f"3. margin: {self.margin_balance}\n"
            f"4. free: {self.free_balance}\n"
            f"5. lock: {self.lock_balance}\n"
            f"6. un_pnl: {self.unrealized_pnl_balance:,.2f}\n"
            f"7. pnl: {self.pnl_balance:,.2f}\n"
            f"8. pnl ratio: {self.pnl_ratio * 100:,.2f} %\n"
        )

In [11]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

# 예제 실행
q_ = Queue()
init_balance = 1000
api_key = base_utils.load_json(Path.bianace)
futures_tr_client_instance = futures_tr_client(**api_key)  # 클라이언트 인스턴스 생성

# 올바르게 Wallet 인스턴스 생성
ins_wallet = Wallet(init_balance=init_balance, queue=q_, futures_trading_client=futures_tr_client_instance)

async def run():
    p1 = asyncio.create_task(ins_wallet.run_update())
    p2 = asyncio.create_task(ins_wallet.send_queue())
    await asyncio.gather(p2, p1)

asyncio.run(run())

 0 회차 

1. seed money: 1000
2. total balance: 3.43517474
3. margin: 3.43517474
4. free: 1.2346635
5. lock: 0.0
6. un_pnl: 0.54162706
7. pnl: 0.54162706
8. pnl ratio: 0.05 %

 1 회차 

1. seed money: 1000
2. total balance: 3.43517474
3. margin: 3.43517474
4. free: 1.2346635
5. lock: 0.0
6. un_pnl: 0.54162706
7. pnl: 0.54162706
8. pnl ratio: 0.05 %

 2 회차 

1. seed money: 1000
2. total balance: 3.45303348
3. margin: 3.45303348
4. free: 1.25291047
5. lock: 0.0
6. un_pnl: 0.5594858
7. pnl: 0.5594858
8. pnl ratio: 0.06 %

 3 회차 

1. seed money: 1000
2. total balance: 3.46504768
3. margin: 3.46504768
4. free: 1.26518585
5. lock: 0.0
6. un_pnl: 0.5715
7. pnl: 0.5715
8. pnl ratio: 0.06 %

 4 회차 

1. seed money: 1000
2. total balance: 3.47774768
3. margin: 3.47774768
4. free: 1.27816193
5. lock: 0.0
6. un_pnl: 0.5842
7. pnl: 0.5842
8. pnl ratio: 0.06 %



KeyboardInterrupt: 

In [ ]:
asyncio.run(ins_wallet.run_update())